In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
são_paulo_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/SP_PROCESSO_E_ASSUNTO_FEITOS_BCKPV2.json')    

In [7]:
são_paulo_sample = [são_paulo_df.loc[114,'Conteúdo']]
são_paulo_sample

['0042934-50.2012.8.26.0196 (196.01.2012.042934-1/000000-000) Nº Ordem: 002792/2012 - Divórcio Litigioso - Dissolução - N. M. V. S. X R. D. S. - Expeça-se certidão de honorários (R$654,75 - cód 203) e arquivem-se os autos. Intimem-se. Ciência ao Ministério Público. - ADV SILVANA DE ANDRADE PRADO OAB/SP 175220 - ADV NILCILENE REIS MAXIMIANO DO NASCIMENTO OAB/SP 182011']

In [8]:
são_paulo_sample_tokenized = tok.texts_to_sequences(são_paulo_sample)
pd.DataFrame(são_paulo_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,180020,363,58,15,17,1721,1451,51,58,180021,...,120818,18,180023,890,12733,98,1180,10,11,180024


In [9]:
são_paulo_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=são_paulo_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(são_paulo_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,180020,363,58,15,17,1721,1451,51,58,180021,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predicted_são_paulo_sample_tokenized_padded = model(são_paulo_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_são_paulo_sample_tokenized_padded[0]):
    if (i < len(são_paulo_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[são_paulo_sample_tokenized[0][i]])

1 0042934
1 50
1 2012
1 8
1 26
1 0196
1 196
1 01
1 2012
1 042934
1 1
1 000000
1 000
1 Nº
1 Ordem
1 002792
1 2012
14 Divórcio
2 Litigioso
13 Dissolução
1 N
1 M
1 V
1 S
1 X
1 R
1 D
1 S
1 Expeça
1 se
1 certidão
1 de
1 honorários
1 R$654
1 75
1 cód
1 203
1 e
1 arquivem
1 se
1 os
1 autos
1 Intimem
1 se
1 Ciência
1 ao
1 Ministério
1 Público
1 ADV
1 SILVANA
1 DE
1 ANDRADE
1 PRADO
1 OAB
1 SP
1 175220
1 ADV
1 NILCILENE
1 REIS
1 MAXIMIANO
1 DO
1 NASCIMENTO
1 OAB
1 SP
1 182011
